 # Análisis y Visualización — Incendios en España

 Este cuaderno carga el dataset **procesado** y genera visualizaciones:
 - Tendencia anual de hectáreas quemadas
 - Recursos utilizados por año (barras apiladas)
 - Top-N provincias por hectáreas quemadas

 **Autores:** Lorena Elena Mohanu · José Ancízar Arbeláez Nieto
 **Licencia:** MIT (ver `LICENSE` en el repositorio)

 ## Importaciones

In [7]:
# --- Configuración robusta de rutas para importar `wildfires` desde notebooks/ ---
from pathlib import Path
import sys

cwd = Path.cwd().resolve()
candidates = [cwd, cwd.parent, cwd.parent.parent]  # ., .., ../..
added = False

for base in candidates:
    src = base / "src"
    if (src / "wildfires").exists():
        sys.path.insert(0, str(src))
        added = True
        break

if not added:
    raise RuntimeError(
        "No se encontró 'src/wildfires' en ., .. o ../..  "
        "Asegúrate de ejecutar el notebook dentro del repo."
    )

# Prueba de import
import wildfires
print("wildfires importado desde:", wildfires.__file__)

wildfires importado desde: C:\Users\loren\Downloads\Semestre 5\visualizacion_de_datos\vdat-p3-GJ01\Proyecto_visualizacion_incendios\src\wildfires\__init__.py


In [8]:
import pandas as pd

# `display` para notebooks; si ejecutas como script, lo redefinimos a print
try:
    from IPython.display import display
except Exception:  # pragma: no cover
    def display(x):  # type: ignore
        print(x)

from wildfires import (
    add_total_resources,
    aggregate_burned_area_by_year, aggregate_resources_by_year, top_provinces_by_burned_area,
    line_hectares_by_year, stacked_resources_by_year, horizontal_top_provinces
)

 ## Parámetros

In [9]:
# Ruta del parquet generado en el cuaderno de preparación
PARQUET_PATH = "../data/processed/incendios_clean.parquet"

# Tamaño del Top-N de provincias para el ranking
TOP_N = 10

# Mapa opcional para renombrar leyendas de recursos en las barras apiladas
RENAME_MAP = {
    "numeromediospersonal": "Personal",
    "numeromediospesados": "Pesados",
    "numeromediosaereos": "Aéreos",
}

 ## Carga de datos procesados

In [10]:
df = pd.read_parquet(PARQUET_PATH)
print("Filas:", len(df))
display(df.head(5))

Filas: 585399


,anio,idpeligro,idprovincia,provincia,numeromediospersonal,numeromediospesados,numeromediosaereos,perdidassuperficiales,idcausa,intencionado,hectareas_quemadas
0,2016,-1,7,Illes Balears,20,18,0,0.20,290,False,0.20
1,2016,-1,22,Huesca,4,0,0,0.05,322,False,0.05
2,2016,-1,22,Huesca,9,4,0,0.23,241,False,0.23
3,2016,-1,22,Huesca,8,4,0,0.65,212,False,0.65
4,2016,-1,22,Huesca,7,3,0,0.15,212,False,0.15


 ## Resumen temporal

In [11]:
if "anio" in df.columns and not df.empty:
    years = sorted(df["anio"].dropna().astype(int).unique().tolist())
    rango = (years[0], years[-1]) if years else None
else:
    years, rango = [], None

print("Rango de años:", rango)

Rango de años: (1968, 2016)


 ## Tendencia de hectáreas quemadas por año

In [12]:
# Figura de línea (usa alias 'hectareas_quemadas' si está disponible)
fig_line = line_hectares_by_year(df)
fig_line

 ## Recursos de extinción por año (barras apiladas)

In [13]:
# Agregación + figura de barras apiladas con leyendas legibles
fig_stack = stacked_resources_by_year(df, rename_map=RENAME_MAP)
fig_stack

 ## Top provincias por hectáreas quemadas

In [14]:
# Tabla Top-N (agregado por provincia)
topN = top_provinces_by_burned_area(df, n=TOP_N)
display(topN)

# Gráfico de barras horizontales con el mismo Top-N
fig_top = horizontal_top_provinces(df, n=TOP_N)
fig_top

,provincia,hectareas_quemadas
35,Ourense,659287.50
28,León,571544.25
5,Asturias,512377.51
37,Pontevedra,433536.97
15,Cáceres,426283.70
47,València/Valencia,406840.47
30,Lugo,405506.33
0,A Coruña,363661.72
48,Zamora,346840.95
10,Cantabria,227315.61
